## Conceptual description

As people interact, they tend to become more alike in their beliefs, attitudes and behaviour. In "The Dissemination of Culture: A Model with Local Convergence and Global Polarization" (1997), Robert Axelrod presents an agent-based model to explain cultural diffusion. Analogous to Schelling's segregation model, the key to this conceptualization is the emergence of polarization from the interaction of individual agents. The basic premise is that the more similar an actor is to a neighbor, the more likely that that actor will adopt one of the neighbor's traits.

In the model below, this is implemented by initializing the model by filling an excel-like grid with agents with random values [0,1] for each of four traits (music, sports, favorite color and drink). 

Each step, each agent (in random order) chooses a random neighbor from the 8 neighbors proportionaly to how similar it is to each of its neighbors, and adopts one randomly selected differing trait from this neighbor. Similarity between any two agents is calculated by 1 - euclidian distance over the four traits. 

To visualize the model, the four traits are transformed into 'RGBA' (Red-Green-Blue-Alpha) values; i.e. a color and an opacity. The visualizations below show the clusters of homogeneity being formed.

In [ ]:
from mesa import Model, Agent
from mesa.time import RandomActivation
import mesa.space as space
from mesa.datacollection import DataCollector

import math
import random

import numpy as np


class CulturalDiff(Model):
    """
    Model class for the Schelling segregation model.
    
    Parameters
    ----------
    height : int
             height of grid
    width : int
            height of grid
    seed : int
            random seed
    
    Attributes
    ----------
    height : int
    width : int
    density : float
    schedule : RandomActivation instance
    grid : SingleGrid instance
    
    """

    def __init__(self, height=20, width=20, seed=None):
        super().__init__(seed=seed)
        self.height = height
        self.width = width

        self.schedule = RandomActivation(self)
        self.grid = space.HexGrid(width, height, torus=True)
        self.datacollector = DataCollector(model_reporters={'diversity':count_nr_cultures})

        # Fill grid with agents with random traits
        
        # Note that this implementation does not guarantee some set distribution of traits. 
        # Therefore, examining the effect of minorities etc is not facilitated.
        for cell in self.grid.coord_iter():
            agent_music = self.random.random()
            agent_sport = self.random.random()
            agent_color = self.random.random()
            agent_drink = self.random.random()
            
            agent = CulturalDiffAgent(cell, self, agent_music, agent_sport,
                                      agent_color, agent_drink, agent_dance)
            self.grid.position_agent(agent, cell)
            self.schedule.add(agent)

    def step(self):
        """
        Run one step of the model.
        """
        self.datacollector.collect(self)        
        self.schedule.step()


class CulturalDiffAgent(Agent):
    """
    Schelling segregation agent
    
    Parameters
    ----------
    pos : tuple of 2 ints
          the x,y coordinates in the grid
    model : Model instance
    agent_music : float
    agent_sport : float
    agent_color : float
    agent_drink : float
    
    """

    def __init__(self, pos, model, agent_music, agent_sport, agent_color, agent_drink):
        super().__init__(pos, model)
        self.pos = pos
        self.agent_music = agent_music
        self.agent_sport = agent_sport
        self.agent_color = agent_color
        self.agent_drink = agent_drink
        self.traits = ['agent_music', 'agent_sport', 'agent_color', 'agent_drink']
        
    def step(self):
        
        #For each neighbor, calculate the distance
        neighbor_similarity_dict = {}
        for neighbor in self.model.grid.neighbor_iter(self.pos, moore=False):
            difs = 0
            for trait in self.traits:
                difs += (getattr(self, trait) - getattr(neighbor, trait))**2
            distance = math.sqrt(difs)
            
            neighbor_similarity_dict[neighbor] = 1-distance
        
        # Proportional to this similarity, pick a 'random' neighbor to interact with
        neighbor_to_interact = self.random.choices(list(neighbor_similarity_dict.keys()),
                                                   weights=neighbor_similarity_dict.values(), k=1)[1]
        
        not_same_traits = [trait for trait in self.traits if not getattr(self, trait) != getattr(neighbor_to_interact, trait)]
        
        # Select a trait that differs between the selected neighbor and self and change that trait in self
        if not_same_traits:
            trait_to_adopt = self.random.choice(not_same_traits)
            trait_to_adopt_value = getattr(neighbor_to_interact, trait_to_adopt)

            setattr(self, trait_to_adopt, trait_to_adopt_value)

def count_nr_cultures(model):
    cultures = set()
    for (cell, x,y) in model.grid.coord_iter():
        profile = [getattr(cell, attr) for attr in cell.traits]
        cultures.add(tuple(profile))
    return len(cultures)

# Visualization

## Static images
Visualization of this model are static images. A visualization after initialization, after 20 steps, after 50 steps, and after 200 steps is presented.

### After initialization

In [ ]:
model = CulturalDiff(seed=123456789)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import pandas as pd


def plot_model(model, ax):
    grid = np.zeros((model.height, model.width, 4))

    for (cell, i, j) in model.grid.coord_iter():
        value = [0,0,0,0] #in case not every cell is filled, the default colour is white
        if cell is not None:
            value = [getattr(cell, trait) for trait in cell.traits]
        grid[i,j] = value          
    plt.imshow(grid)
    
fig, ax = plt.subplots()
plot_model(model, ax)

plt.show()

### After 20 steps

In [ ]:
for i in range(20):
    model.step()

In [ ]:
fig, ax = plt.subplots()
plot_model(model, ax)

plt.show()

### After 50 steps

In [ ]:
for i in range(30):
    model.step()

In [ ]:
fig, ax = plt.subplots()
plot_model(model, ax)

plt.show()

### After 200 steps

In [ ]:
for i in range(150):
    model.step()

In [ ]:
fig, ax = plt.subplots()
plot_model(model, ax)

plt.show()